# 📢📢📢 Projet NLP : La parole aux citoyens (Make.org - Le tourisme vert en Ille-et-Vilaine)📢📢📢
# Preprocessing NLP
## Bibliothèques nécessaires

In [1]:
import os

import nltk
import pandas as pd
import spacy
from gensim.utils import simple_preprocess
from nltk.data import find

## Téléchargements des utilitaires si nécessaire

In [2]:
# 🔧 Dossier local de ressources NLTK (dans ton projet ou venv)
NLTK_LOCAL_PATH = "../src/nltk_data"

# 🔄 Ajouter ce dossier au chemin de recherche
nltk.data.path.append(NLTK_LOCAL_PATH)

In [3]:
# 📥 Fonction utilitaire pour télécharger uniquement si nécessaire
def safe_nltk_download(resource_name, download_dir=NLTK_LOCAL_PATH):
    try:
        find(resource_name)
    except LookupError:
        nltk.download(resource_name, download_dir)

In [4]:
# 📚 Téléchargements nécessaires
safe_nltk_download("punkt")  # Tokenizer
safe_nltk_download("punkt_tab")  # Tokenizer
safe_nltk_download("averaged_perceptron_tagger")  # POS tagger anglais
safe_nltk_download("universal_tagset")  # Universal tag mapping
safe_nltk_download("stopwords")  # Stopwords

[nltk_data] Downloading package punkt to ../src/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to ../src/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     ../src/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     ../src/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package stopwords to ../src/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words("french"))  # Charger les stop words pour le français
nlp = spacy.load("fr_core_news_sm")  # Chargement du modèle de langue française de SpaCy

## Lecture des données

In [6]:
df = pd.read_csv(
    "../data/processed/req_tourisme_responsable.csv", delimiter=";", encoding="utf-8"
)

df.head(3)

,content,agree_count,disagree_count,neutral_count,agree_score,disagree_score,neutral_score,doNotCare,doNotUnderstand,doable,impossible,likeIt,noOpinion,noWay,platitudeAgree,platitudeDisagree,nb_interrogations,nb_exclamation
0,Il faut une surveillance des lieux touristique...,27,7,7,0.65,0.18,0.17,1,0,8,2,2,0,3,3,4,0,0
1,Il faut donner des sacs poubelles et des cendr...,56,18,15,0.62,0.19,0.19,0,0,21,6,6,5,4,5,7,0,0
2,Il faut récompenser et mettre en avant les act...,35,7,14,0.62,0.13,0.25,0,5,12,2,2,2,3,1,5,0,0


## Preprocessing
- Tokenisation avec NLTK (obtenir un decoupage pour chaque mot) pour garder les mots en majuscules
- Tokenisation avec GENSIM en utilisant simple preprocess pour normaliser (enlever les accents, mettre en minuscules, ...)
### Avec NLTK

In [7]:
# Tokeniser chaque document dans la colonne 'texte'
df["tokens"] = df["content"].apply(lambda x: word_tokenize(x, language="french"))

In [8]:
df["tokens"].head()

0    [Il, faut, une, surveillance, des, lieux, tour...
1    [Il, faut, donner, des, sacs, poubelles, et, d...
2    [Il, faut, récompenser, et, mettre, en, avant,...
3    [Il, faut, promouvoir, toutes, les, destinatio...
4    [Il, faut, enrayer, la, spéculation, immobiliè...
Name: tokens, dtype: object

In [9]:
# Créer la colonne 'case' en vérifiant si chaque token est en majuscules ou en minuscules
df["case"] = df["tokens"].apply(
    lambda tokens: ["Maj" if token.isupper() else "Min" for token in tokens]
)

### Avec GENSIM

In [10]:
# Tokeniser chaque document dans la colonne 'texte'
df["tokens_normalized"] = df["content"].apply(
    lambda x: simple_preprocess(x, deacc=True)
)

In [11]:
df[["case", "tokens_normalized"]].head()

,case,tokens_normalized
0,"[Min, Min, Min, Min, Min, Min, Min, Min, Min, ...","[il, faut, une, surveillance, des, lieux, tour..."
1,"[Min, Min, Min, Min, Min, Min, Min, Min, Min, ...","[il, faut, donner, des, sacs, poubelles, et, d..."
2,"[Min, Min, Min, Min, Min, Min, Min, Min, Min, ...","[il, faut, recompenser, et, mettre, en, avant,..."
3,"[Min, Min, Min, Min, Min, Min, Min, Min, Min, ...","[il, faut, promouvoir, toutes, les, destinatio..."
4,"[Min, Min, Min, Min, Min, Min, Min, Min, Min, ...","[il, faut, enrayer, la, speculation, immobilie..."


## Traitement des stop words

In [12]:
# Enlever les stop words des tokens normalisés et stocker les mots retirés dans une nouvelle colonne
df["stop_words"] = df["tokens_normalized"].apply(
    lambda tokens: [token for token in tokens if token in stop_words]
)

In [13]:
df[["tokens_normalized", "stop_words"]].head()

,tokens_normalized,stop_words
0,"[il, faut, une, surveillance, des, lieux, tour...","[il, une, des, avec, la, pour, ou]"
1,"[il, faut, donner, des, sacs, poubelles, et, d...","[il, des, et, des, de, aux, des, et, en, de]"
2,"[il, faut, recompenser, et, mettre, en, avant,...","[il, et, en, les, des, du, et, des, pour]"
3,"[il, faut, promouvoir, toutes, les, destinatio...","[il, les, et, le]"
4,"[il, faut, enrayer, la, speculation, immobilie...","[il, la, en, de, pour, un]"


In [14]:
# Enlever les deux premiers termes de chaque liste dans la colonne 'tokens_normalized'
df["tokens_normalized"] = df["tokens_normalized"].apply(lambda tokens: tokens[2:])

In [15]:
df["tokens_normalized"].head()

0    [une, surveillance, des, lieux, touristiques, ...
1    [donner, des, sacs, poubelles, et, des, cendri...
2    [recompenser, et, mettre, en, avant, les, acte...
3    [promouvoir, toutes, les, destinations, et, de...
4    [enrayer, la, speculation, immobiliere, en, te...
Name: tokens_normalized, dtype: object

In [16]:
# Enlever les stop words des tokens normalisés
df["tokens_final"] = df["tokens_normalized"].apply(
    lambda tokens: [token for token in tokens if token not in stop_words]
)

In [17]:
df["tokens_final"].head()

0    [surveillance, lieux, touristiques, amende, cl...
1    [donner, sacs, poubelles, cendriers, poche, va...
2    [recompenser, mettre, avant, actes, projets, e...
3    [promouvoir, toutes, destinations, developper,...
4    [enrayer, speculation, immobiliere, territoire...
Name: tokens_final, dtype: object

## Lemmatisation avec spaCy

In [18]:
# Fonction pour lemmatiser les tokens
def lemmatize_tokens(tokens):
    # Joindre les tokens en une chaîne de caractères
    text = " ".join(tokens)
    # L'objet nlp traite le texte
    doc = nlp(text)
    # Lemmatiser chaque token
    lemmatized_tokens = [token.lemma_ for token in doc]
    return lemmatized_tokens

In [19]:
# Appliquer la lemmatisation à chaque ligne de la colonne 'tokens_normalized'
df["tokens_lemmatized"] = df["tokens_final"].apply(lambda x: lemmatize_tokens(x))

In [20]:
df["tokens_lemmatized"].head()

0    [surveillance, lieux, touristique, amende, cle...
1    [donner, sac, poubelle, cendrier, poch, vacanc...
2    [recompenser, mettre, avant, acte, projet, eco...
3    [promouvoir, tout, destination, developper, to...
4    [enrayer, speculation, immobilier, territoire,...
Name: tokens_lemmatized, dtype: object

## Suite à plusieurs visualisations de nuage de mots, enrichissement des stop_words
- les dérive de tourisme
- tout
- faire
- plus

Ces mots n'apportent pas grand-chose sur le sujet. Donc on les enlève des tokens lemmatisés. Bien sûr, ce processus pourrait continuer de manière itérative, mais il faut savoir s'arrêter.

🔍 Pourquoi j'ai choisi d'enlever les stop words avant vectorisation ?
Les stop words (comme le, la, de, et, que...) :
- n’apportent pas d'information sémantique forte ;
- sont trop fréquents → ils polluent l’apprentissage de Word2Vec ;
- génèrent des vecteurs très proches entre eux, peu utiles pour des tâches de similarité ou de clustering.

Word2Vec apprend à prédire un mot à partir du contexte (ou l’inverse). Les stop words, très fréquents, apparaissent dans presque tous les contextes → ils biaisent l’apprentissage.

In [21]:
stop_words.add("tourisme")
stop_words.add("touristique")
stop_words.add("touriste")
stop_words.add("tourism")
stop_words.add("tout")
stop_words.add("plus")
stop_words.add("faire")

In [22]:
# Enlever les stop words des tokens normalisés
df["tokens_final"] = df["tokens_lemmatized"].apply(
    lambda tokens: [token for token in tokens if token not in stop_words]
)

## Sauvegarde du dataframe

In [ ]:
# Repertoire de sauvegarde
savepath = "../data/processed"

# Test de l'existence du répertoire
if not os.path.exists(savepath):
    # Création du répertoire s'il n'existe pas
    os.makedirs(savepath)

# Sauvegarde des jeux de données
df.to_csv(
    path_or_buf=os.path.join(savepath, "req_tourisme_responsable_tokenised.csv"),
    sep=";",
    encoding="utf-8",
    index_label=False,
)